#Depression Detecting Machine Learning



#### 1. Normal
#### 2. Depression level 1
#### 3. Depression level 2

### Importing Libraries

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
combined=pd.read_csv('/content/drive/MyDrive/combined-set.csv')

In [ ]:
nond=pd.read_csv('/content/drive/MyDrive/final.csv')

In [ ]:
suicide=pd.read_csv('/content/drive/MyDrive/Suicide_Detection.csv')

## Preprocessing Dataset

In [ ]:
suicide=suicide.drop('Unnamed: 0',axis=1) # droping unneccesary columns

In [ ]:
nond=nond.drop('Unnamed: 0',axis=1) #dropping unneccesary columns

In [ ]:
suicide.rename(columns = {'class':'label'}, inplace = True) #renaming the columns

In [ ]:
suicide['label'].mask(suicide['label'] == 'suicide', 2, inplace=True) #Changing output columns values

In [ ]:
suicide.drop(suicide[(suicide['label'] =='non-suicide')].index, inplace=True)

In [ ]:
suicide.head(5)

,text,label
0,Ex Wife Threatening SuicideRecently I left my ...,2
3,i need helpjust help me im crying so hard,2
4,"I’m so lostHello, my name is Adam (16) and I’v...",2
5,Honetly idkI dont know what im even doing here...,2
6,[Trigger warning] Excuse for self inflicted bu...,2


In [ ]:
nond.head(5)

,text,label
0,the real reason why you be sad you be attach t...,1
1,my biggest problem be overthinking everything,1
2,the worst sadness be the sadness you have teac...,1
3,i cannot make you understand i cannot make any...,1
4,i do not think anyone really understand how ti...,1


In [ ]:
nond.isnull().sum() #checking any null values

text     0
label    0
dtype: int64

In [ ]:
combined.head(5)

,is_suicide,selftext_clean
0,0,hi really know phrase situation try life reall...
1,1,depressed past couple week ever since got back...
2,0,hi know else go devastated right feeling like ...
3,1,shit like get better everyone purpose need fin...
4,0,wish wa prettier wish feel like burden wish br...


In [ ]:
combined['is_suicide'].mask(combined['is_suicide'] == '1', 2, inplace=True) #replacing output labels

In [ ]:
combined['is_suicide'].mask(combined['is_suicide'] == '0', 1, inplace=True)

In [ ]:
combined.head(5)

,is_suicide,selftext_clean
0,1,hi really know phrase situation try life reall...
1,2,depressed past couple week ever since got back...
2,1,hi know else go devastated right feeling like ...
3,2,shit like get better everyone purpose need fin...
4,1,wish wa prettier wish feel like burden wish br...


In [ ]:
combined.rename(columns = {'selftext_clean':'text'}, inplace = True) # renaming columns
combined.rename(columns={'is_suicide':'label'},inplace=True)

In [ ]:
frames = [nond,combined,suicide]  #combining all three csv files

df = pd.concat(frames) #concating files

In [ ]:
df.drop(df[(df['label'] =='hi')].index, inplace=True) #dropping unnecessary values

In [ ]:
df['label'].value_counts() #no of labels in output class

2    117015
0      8570
1      7705
Name: label, dtype: int64

In [ ]:
df.duplicated().sum() # dropping duplicate values
df.drop_duplicates(inplace=True)
df=df.dropna(how='all')

In [ ]:
df.dtypes

text     object
label    object
dtype: object

In [ ]:
df['text'] = df['text'].replace(np.nan, '')

<ipython-input-24-b5b43d0812f7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].replace(np.nan, '')


In [ ]:
df.isnull().sum()

text     0
label    0
dtype: int64

## UnderSampling

Undersampling is a technique to balance uneven datasets by keeping all of the data in the minority class and decreasing the size of the majority class.

RandomUnderSampler is a fast and easy way to balance the data by randomly selecting a subset of data for the targeted classes. Under-sample the majority class(es) by randomly picking samples with or without replacement.

In [ ]:
X = df.iloc[:,0:1] #diving it text and labels
y = df['label']

In [ ]:
y.value_counts()

2    116949
0      8358
1      7567
Name: label, dtype: int64

In [ ]:
!pip install Counter

In [ ]:
from collections import Counter

In [ ]:
!pip install imblearn

from imblearn import under_sampling

In [ ]:
y = y.astype(int)

In [ ]:
# random undersampling

from imblearn.under_sampling import RandomUnderSampler
rus=RandomUnderSampler(random_state=0)
X_resampled,y_resampled=rus.fit_resample(X,y)
print(sorted(Counter(y_resampled).items()),y_resampled.shape)

[(0, 7567), (1, 7567), (2, 7567)] (22701,)


In [ ]:
X_resampled.shape,y_resampled.shape # total we have 22,701 rows

((22701, 1), (22701,))

In [ ]:
X_resampled.head()

,text
0,yo twitter what be her i be tryna end up like ...
1,you literally sell cars that do not exist yet
2,a defective choice be not a reflection of the ...
3,it s 0am i ve been awake for nearly hour
4,my dads face after see his first sexy superher...


In [ ]:
y_resampled

0        0
1        0
2        0
3        0
4        0
        ..
22696    2
22697    2
22698    2
22699    2
22700    2
Name: label, Length: 22701, dtype: int64

In [ ]:
y_new=pd.DataFrame(y_resampled)

In [ ]:
y_new

,label
0,0
1,0
2,0
3,0
4,0
...,...
22696,2
22697,2
22698,2
22699,2


#Text Preprocessing

In [ ]:
X_resampled['text'] = X_resampled['text'].apply(lambda x:x.lower()) #All text to lowecase

In [ ]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*!?>'), '', raw_text) #removing html tags
    return cleaned_text

In [ ]:
X_resampled['text'] = X_resampled['text'].apply(remove_tags)

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Stop word removal is one of the most commonly used preprocessing steps across different NLP applications. The idea is simply removing the words that occur commonly across all the documents in the corpus

In [ ]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

X_resampled['text'] = X_resampled['text'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [ ]:
X_resampled

,text
0,yo twitter tryna end like cuties back
1,literally sell cars exist yet
2,defective choice reflection truth see manifest...
3,0am awake nearly hour
4,dads face see first sexy superhero cosplay
...,...
22696,life net negativelife always seemed baseline z...
22697,"eating disorder ruining lifei’m 17, eating dis..."
22698,need someone talk to... train approaching toda...
22699,fuck itthe anniversary girlfriend dying sunday...


In [ ]:
#Removing puntuactions
import string
exclude=string.punctuation
def remove_punc(text):
  for char in exclude:
    text=text.replace(char,'')
  return text


X_resampled['text'].apply(remove_punc)

0                    yo twitter tryna end like cuties back
1                            literally sell cars exist yet
2        defective choice reflection truth see manifest...
3                                    0am awake nearly hour
4               dads face see first sexy superhero cosplay
                               ...                        
22696    life net negativelife always seemed baseline z...
22697    eating disorder ruining lifei’m 17 eating diso...
22698    need someone talk to train approaching today t...
22699    fuck itthe anniversary girlfriend dying sunday...
22700    insult injurylast week moment severe pain post...
Name: text, Length: 22701, dtype: object

In [ ]:
# cleaning Short Forms in Social Media  Tweets
chat_words={'AFAIK':'As Far As I Know',
'AFK':'Away From Keyboard',
'ASAP':'As Soon As Possible',
'ATK':'At The Keyboard',
'ATM':'At The Moment',
'A3':'Anytime, Anywhere, Anyplace',
'BAK':'Back At Keyboard',
'BBL':'Be Back Later',
'BBS':'Be Back Soon',
'BFN':'Bye For Now',
'B4N':'Bye For Now',
'BRB':'Be Right Back',
'BRT':'Be Right There',
'BTW':'By The Way',
'B4':'Before',
'B4N':'Bye For Now',
'CU':'See You',
'CUL8R':'See You Later',
'CYA':'See You',
'FAQ':'Frequently Asked Questions',
'FC':'Fingers Crossed',
'FWIW':'For What Its Worth',
'FYI':'For Your Information',
'GAL':'Get A Life',
'GG':'Good Game',
'GN':'Good Night',
'GMTA':'Great Minds Think Alike',
'GR8':'Great!',
'G9':'Genius',
'IC':'I See',
'ICQ':'I Seek you',
'ILU':'I Love You',
'IMHO':'In My Honest/Humble Opinion',
'IMO':'In My Opinion',
'IOW':'In Other Words',
'IRL':'In Real Life',
'KISS':'Keep It Simple',
'LDR':'Long Distance Relationship',
'LMAO':'Laugh My Ass Off',
'LOL':'Laughing Out Loud',
'LTNS':'Long Time No See',
'L8R':'Later',
'MTE':'My Thoughts Exactly',
'M8':'Mate',
'NRN':'No Reply Necessary',
'OIC':'Oh I See',
'PITA':'Pain In The Ass',
'PRT':'Party',
'PRW':'Parents Are Watching',
'ROFL':'Rolling On The Floor Laughing',
'ROFLOL':'Rolling On The Floor Laughing Out Loud',
'ROTFLMAO':'Rolling On The Floor Laughing My Ass Off',
'SK8':'Skate',
'THX':'Thank You',
'TTFN':'Ta-Ta For Now',
'TTYL':'Talk To You Later',
'U':'You',
'U2':'You Too',
'U4E':'Yours For Ever',
'WB':'Welcome Back',
'WTF':'What The Fuck',
'WTG':'Way To Go',
'WUF':'Where Are You From',
'W8':'Wait',
}

In [ ]:
chat_words

{'AFAIK': 'As Far As I Know',
 'AFK': 'Away From Keyboard',
 'ASAP': 'As Soon As Possible',
 'ATK': 'At The Keyboard',
 'ATM': 'At The Moment',
 'A3': 'Anytime, Anywhere, Anyplace',
 'BAK': 'Back At Keyboard',
 'BBL': 'Be Back Later',
 'BBS': 'Be Back Soon',
 'BFN': 'Bye For Now',
 'B4N': 'Bye For Now',
 'BRB': 'Be Right Back',
 'BRT': 'Be Right There',
 'BTW': 'By The Way',
 'B4': 'Before',
 'CU': 'See You',
 'CUL8R': 'See You Later',
 'CYA': 'See You',
 'FAQ': 'Frequently Asked Questions',
 'FC': 'Fingers Crossed',
 'FWIW': 'For What Its Worth',
 'FYI': 'For Your Information',
 'GAL': 'Get A Life',
 'GG': 'Good Game',
 'GN': 'Good Night',
 'GMTA': 'Great Minds Think Alike',
 'GR8': 'Great!',
 'G9': 'Genius',
 'IC': 'I See',
 'ICQ': 'I Seek you',
 'ILU': 'I Love You',
 'IMHO': 'In My Honest/Humble Opinion',
 'IMO': 'In My Opinion',
 'IOW': 'In Other Words',
 'IRL': 'In Real Life',
 'KISS': 'Keep It Simple',
 'LDR': 'Long Distance Relationship',
 'LMAO': 'Laugh My Ass Off',
 'LOL': 'La

In [ ]:
def chat_conversion(text):
  new_text=[]
  for w in text.split():
    if w.upper() in chat_words:
      new_text.append(chat_words[w.upper()])
    else:
      new_text.append(w)
  return ' '.join(new_text)


X_resampled['text'].apply(chat_conversion)

0                    yo twitter tryna end like cuties back
1                            literally sell cars exist yet
2        defective choice reflection truth see manifest...
3                                    0am awake nearly hour
4               dads face see first sexy superhero cosplay
                               ...                        
22696    life net negativelife always seemed baseline z...
22697    eating disorder ruining lifei’m 17, eating dis...
22698    need someone talk to... train approaching toda...
22699    fuck itthe anniversary girlfriend dying sunday...
22700    insult injurylast week, moment severe pain, po...
Name: text, Length: 22701, dtype: object

###Tokenization and stop word removal

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))





[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
tokenized_sentences = []
for sentence in  X_resampled['text']:
    tokenized_sentence = word_tokenize(sentence)
    filtered_sentence = [word for word in tokenized_sentence if word.lower() not in stop_words]
    tokenized_sentences.append(filtered_sentence)


In [ ]:
X_resampled['tokenized']=tokenized_sentences

In [ ]:
X_resampled.head()

,text,tokenized
0,yo twitter tryna end like cuties back,"[yo, twitter, tryna, end, like, cuties, back]"
1,literally sell cars exist yet,"[literally, sell, cars, exist, yet]"
2,defective choice reflection truth see manifest...,"[defective, choice, reflection, truth, see, ma..."
3,0am awake nearly hour,"[0am, awake, nearly, hour]"
4,dads face see first sexy superhero cosplay,"[dads, face, see, first, sexy, superhero, cosp..."


### Text representations using Bow,TFIDF and Glove

In [ ]:
import numpy as np
import pandas as pd
import nltk
import spacy
import os
import string
import gensim


In [ ]:

!wget -c https://nlp.stanford.edu/data/glove.42B.300d.zip


--2023-07-29 06:43:43--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2023-07-29 06:43:44--  https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [ ]:

import zipfile
with zipfile.ZipFile("/content/glove.42B.300d.zip", "r") as zipread:
  zipread.extractall("/content/")
  zipread.close


In [ ]:
import gensim
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec


In [ ]:
 #= "/content/glove.42B.300d.txt"
PATH_TO_GLOVE = datapath("/content/glove.42B.300d.txt")

#tmp_file = "/glove.840B.300d.w2v.txt"
tmp_file = get_tmpfile("glove.42B.300d.w2v.txt")
glove2word2vec(PATH_TO_GLOVE, tmp_file)
glove = gensim.models.KeyedVectors.load_word2vec_format(tmp_file, binary = False)


<ipython-input-56-21ba2147e31e>:6: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(PATH_TO_GLOVE, tmp_file)


In [ ]:
def glove_embedding(text):
  vector_1 = np.mean([glove[word] for word in str(text) if word in glove],axis=0)
  return vector_1


In [ ]:
X_resampled.head()

,text,tokenized
0,yo twitter tryna end like cuties back,"[yo, twitter, tryna, end, like, cuties, back]"
1,literally sell cars exist yet,"[literally, sell, cars, exist, yet]"
2,defective choice reflection truth see manifest...,"[defective, choice, reflection, truth, see, ma..."
3,0am awake nearly hour,"[0am, awake, nearly, hour]"
4,dads face see first sexy superhero cosplay,"[dads, face, see, first, sexy, superhero, cosp..."


In [ ]:
X_resampled['glove_method']=X_resampled['tokenized'].apply(glove_embedding)

In [ ]:
x_new=X_resampled['glove_method']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x_new,y_resampled,test_size=0.2,random_state=1)

In [ ]:
X_train

15576    [0.17577171, -0.004628026, 0.046714827, 0.2327...
3656     [0.18149132, 0.053523332, 0.09314626, 0.280802...
9536     [0.16570425, -0.025533753, 0.111522004, 0.2701...
1897     [0.16150247, -0.022269955, 0.023547372, 0.2505...
5306     [0.18254006, -0.039586425, 0.14014444, 0.21841...
                               ...                        
10955    [0.17134427, 0.061515164, 0.05078706, 0.253675...
17289    [0.15103154, 0.016515827, 0.032292943, 0.23338...
5192     [0.14775622, 0.006440219, 0.007126469, 0.15557...
12172    [0.15624952, 0.010942394, 0.08196603, 0.245790...
235      [0.1659344, 0.025627825, 0.054643627, 0.232560...
Name: glove_method, Length: 18160, dtype: object

In [ ]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((18160,), (4541,), (18160,), (4541,))

### Training the Datasets with various Machine Learning algorithms

# Ensemble Learning

#Boosting Algorithms

####1.Gradient Boost
####2.ADA Boost
####3.XG Boost



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier # Gradient Boost
clf=GradientBoostingClassifier()
clf.fit(list(X_train),y_train)
y_pred=clf.predict(list(X_test))


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.72      0.82      0.77      1523
           1       0.77      0.70      0.73      1480
           2       0.99      0.93      0.96      1538

    accuracy                           0.82      4541
   macro avg       0.82      0.82      0.82      4541
weighted avg       0.83      0.82      0.82      4541



In [ ]:
from xgboost import XGBClassifier

# Create a XGBoost classifier object
xgb_clf= XGBClassifier()

# Train the classifier on the training data
xgb_clf.fit(list(X_train), y_train)

# Make predictions on the test data
Y_pred_xgb = xgb_clf.predict(list(X_test))

from sklearn.metrics import classification_report
print(classification_report(y_test,Y_pred_xgb))


              precision    recall  f1-score   support

           0       0.74      0.83      0.78      1523
           1       0.78      0.74      0.76      1480
           2       1.00      0.93      0.97      1538

    accuracy                           0.83      4541
   macro avg       0.84      0.83      0.83      4541
weighted avg       0.84      0.83      0.84      4541



In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1)
# Train Adaboost Classifer
ada.fit(list(X_train), y_train)

#Predict the response for test dataset
y_pred_ada = ada.predict(list(X_test))

from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred_ada))

              precision    recall  f1-score   support

           0       0.66      0.67      0.67      1523
           1       0.65      0.64      0.64      1480
           2       0.95      0.93      0.94      1538

    accuracy                           0.75      4541
   macro avg       0.75      0.75      0.75      4541
weighted avg       0.75      0.75      0.75      4541



#Voting Ensemble

* Soft
* Hard

In [ ]:
#we are using three estimators Logistic Regression, KNeighborsClassifier,
# and Random Forest Classifier


from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

clf1=LogisticRegression()
clf2=RandomForestClassifier()
clf3=KNeighborsClassifier()

estimators=[('lr',clf),('rf',clf2),('knn',clf3)]


In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
for estimator in estimators:
    x = cross_val_score(estimator[1],list(X_train),y_train,cv=10,scoring='accuracy')
    print(estimator[0],np.round(np.mean(x),2))

lr 0.82
rf 0.81
knn 0.77


In [ ]:
from sklearn.ensemble import VotingClassifier
#hard voting
vc = VotingClassifier(estimators=estimators,voting='hard')
x = cross_val_score(vc,list(X_train),y_train,cv=10,scoring='accuracy')
print(np.round(np.mean(x),2))

In [ ]:
vc1 = VotingClassifier(estimators=estimators,voting='soft')
x = cross_val_score(vc1,list(X_train),y_train,cv=10,scoring='accuracy')
print(np.round(np.mean(x),2))

In [ ]:
for i in range(1,4): #weighted voting
    for j in range(1,4):
        for k in range(1,4):
            vc = VotingClassifier(estimators=estimators,voting='soft',weights=[i,j,k])
            x = cross_val_score(vc,X,y,cv=10,scoring='accuracy')
            print("for i={},j={},k={}".format(i,j,k),np.round(np.mean(x),2))


In [ ]:
#A Histogram-based Gradient Boosting Classification Tree, very fast for big datasets (n_samples >= 10_000).

from sklearn.ensemble import HistGradientBoostingClassifier
clf_hist_gradient = HistGradientBoostingClassifier().fit(list(X_train), y_train)
clf.hist_gradient(X, y)

#Bagging

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf_random = RandomForestClassifier(max_depth=2, random_state=0)
clf_random.fit(list(X_train), y_train)
y_pred_random=clf_random.predict(list(X_test))
print(classification_report(y_test,y_pred_random))


              precision    recall  f1-score   support

           0       0.64      0.68      0.66      1523
           1       0.63      0.63      0.63      1480
           2       0.93      0.88      0.90      1538

    accuracy                           0.73      4541
   macro avg       0.73      0.73      0.73      4541
weighted avg       0.74      0.73      0.73      4541



#Optimization